In [38]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    device_name = tf.test.gpu_device_name()
else:
    device_name = 'CPU'
device_name = 'CPU' 
print(f'device name {device_name}')
import os
import sys

from math import cos,sin,tan,asin,acos,radians,sqrt,degrees,atan,atan2,copysign
import numpy as np

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pickle
import scipy
from scipy.stats import norm
import random
import time
import timeit
import math
import localization as lx
import gzip

import util.npose_util as nu
import datetime
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial import cKDTree


import joblib
from sklearn.manifold import MDS
import argparse
from functools import partial
from itertools import starmap,repeat,permutations

from pymol import cmd, stored, selector

import GenerateEndpoints as ge
import HelixFit as hf
import FitTransform as ft

import seaborn as sns
import util.RotationMethods as rm

import copy




device name CPU


In [29]:
import Guided_Midpoint_Buttress as gmp

In [3]:
start_hel, gep= gmp.get_reference_input(batch=1000 )

In [4]:
out_ep = gmp.build_protein_on_guide(start_hel, gep, batch=200, 
                                 next_mp_dist = 11,mp_deviation_limit = 5)

max next indices 13
(1000, 8, 3)
Input Size:  100000
Passing Filters:  654
final pass filter 616
max next indices 29
(616, 8, 3)
Input Size:  61600
Passing Filters:  682
final pass filter 430
max next indices 48
(430, 8, 3)
Input Size:  43000
Passing Filters:  793
final pass filter 557
max next indices 67
(557, 8, 3)
Input Size:  55700
Passing Filters:  777
final pass filter 424
max next indices 89
(424, 8, 3)
Input Size:  42400
Passing Filters:  365
final pass filter 112
max next indices 97
(75, 8, 3)
Input Size:  7500
Passing Filters:  166
final pass filter 107


In [5]:
gmp.vp(out_ep[0],gep,max_out=100)

 PyMOL not running, entering library mode (experimental)


In [14]:
import LoopEndpoints as le

In [24]:
end_points = out_ep[0][5]
length_mod=1
dist_cut=6
phiQueryNum=10
neighbors = 10
randMult=10
maxPhi_cut=20
verbose= True

In [77]:
end_points.shape

(24, 3)

In [81]:
bL, iL, phiList, iL, iL, suc= add_loops(end_points, neighbors=10, length_mod=3, dist_cut=6, phiQueryNum=10, randMult=0, maxPhi_cut=20,
              uniquePhi=True, verbose=True)

first helix: #7
first loop: #215
second helix: #197
hnum 1
3 helix: reduced to #20
3 helix: #20
hnum 2
4 helix: reduced to #20
4 helix: #20
hnum 3
5 helix: #2
hnum 4
rn: 1
4 helix: #99
hnum 3
5 helix: #60
hnum 4
6 helix: reduced to #120
6 helix: #120
hnum 5
7 helix: #1
hnum 6
rn: 2
6 helix: reduced to #220
6 helix: #220
hnum 5
7 helix: #24
hnum 6
8 helix: #101
hnum 7
9 helix: reduced to #220
9 helix: #220
hnum 8
10 helix: reduced to #220
10 helix: #220
hnum 9
11 helix: reduced to #220
11 helix: #220
hnum 10
12 helix: reduced to #220
12 helix: #220


In [80]:
def add_loops(end_points, neighbors=10, length_mod=1, dist_cut=6, phiQueryNum=10, randMult=10, maxPhi_cut=20,
              uniquePhi=True, verbose=False):
    """Chains first_helix, first_loop, second helix and the iteratively loop_then_helix to add all loops to endpoints."""
    
    epIn = end_points.copy()
    size = int(len(epIn)/2) #number of helixes represented by helical endpoints
    
   
    bL, iL, epGuide = le.first_helix(epIn,length_mod=length_mod)
    if verbose:
        print(f'first helix: #{iL.shape[0]}')
    
    if iL.shape[0] == 0:
        if verbose:
            print('fail')
        return bL, iL, phiList, iL, iL, False
    
    bL, iL, epGuide, epTrue, phiList, loopFeature, xform_True = le.first_loop(bL, iL, epGuide, 
                                                                neighbors=neighbors, phiQueryNum=phiQueryNum,
                                                                           randMult=randMult, distCut=dist_cut)
    if verbose:
        print(f'first loop: #{iL.shape[0]}')
    if iL.shape[0] == 0:
        if verbose:
            print('fail')
        return bL, iL, phiList, iL, iL, False
    
    hnum=1
    bL, iL, epGuide, epTrue, phiList = le.second_helix(bL, iL, epGuide, epTrue, phiList, hnum, loopFeature, 
                                                   xform_True, length_mod=length_mod, distCut=dist_cut)
    if verbose:
        print(f'second helix: #{iL.shape[0]}')
    
    if iL.shape[0] == 0:
        if verbose:
            print('fail')
        return bL, iL, phiList, iL, iL, False
    
    repeatNumber = 0 
    hnum = 0
    
    bL_s = []
    iL_s = []
    epGuide_s = []
    epTrue_s = []
    phiList_s =  []
    for x in range(0,(size-1)):
        bL_s.append([])
        iL_s.append([])
        epGuide_s.append([])
        epTrue_s.append([])
        phiList_s.append([])
        
    
    while hnum<(size-2):
        hnum += 1
        print('hnum',hnum)
        bL, iL, epGuide, epTrue, phiList = le.next_loop_helix(bL, iL, epGuide, epTrue, phiList, hnum, 
                neighbors=neighbors, phiQueryNum=phiQueryNum, randMult=randMult, distCut=dist_cut,length_mod=length_mod)
       
        bL_s[hnum] = copy.deepcopy(bL)
        iL_s[hnum] = iL.copy()
        epGuide_s[hnum] = epGuide.copy()
        epTrue_s[hnum] = epTrue.copy()
        phiList_s[hnum] = phiList.copy()
        
        if iL.shape[0] == 0:
            
            hnum = hnum - 2
            
            bL = bL_s[hnum] 
            iL = iL_s[hnum] 
            epGuide = epGuide_s[hnum] 
            epTrue = epTrue_s[hnum] 
            phiList = phiList_s[hnum] 
            
            neighbors += 5
            maxPhi_cut += 100
            
            repeatNumber += 1
            print('rn:', repeatNumber)
                
            if repeatNumber == 4:
                if verbose:
                    print('fail2')
                return bL, iL, phiList, iL, iL, False
            
        if iL.shape[0] > maxPhi_cut:
            indexer = le.random_reduce(bL,num_to_keep = maxPhi_cut)
            bL = bL[indexer]
            iL = iL[indexer] 
            epGuide = epGuide[indexer]
            epTrue =  epTrue[indexer]
            phiList = phiList[indexer]
            if verbose:
                print(f'{hnum+2} helix: reduced to #{maxPhi_cut}')
        if verbose:
            print(f'{hnum+2} helix: #{iL.shape[0]}')
            
         
    #only return proteins with a unique set of loops queried from different phi bins
    if uniquePhi:
        phiList, u_indices = np.unique(phiList,axis=0,return_index=True)
        epGuide = epGuide[u_indices]
        epTrue = epTrue[u_indices]
        iL = iL[u_indices]
        bL = bL[u_indices]
    
    return bL, iL, phiList, epGuide, epTrue, True

In [47]:
epIn = end_points.copy()
size = int(len(epIn)/2) #number of helixes represented by helical endpoints


bL, iL, epGuide = le.first_helix(epIn,length_mod=length_mod)
if verbose:
    print(f'first helix: #{iL.shape[0]}')

if iL.shape[0] == 0:
    if verbose:
        print('fail')
    #return bL, iL, phiList, iL, iL, False


bL, iL, epGuide, epTrue, phiList, loopFeature, xform_True = le.first_loop(bL, iL, epGuide, 
                                                                neighbors=neighbors, phiQueryNum=phiQueryNum,
                                                                           randMult=randMult, distCut=dist_cut)
if verbose:
    print(f'first loop: #{iL.shape[0]}')
if iL.shape[0] == 0:
    if verbose:
        print('fail')
    #return bL, iL, phiList, iL, iL, False
    
hnum=1
bL, iL, epGuide, epTrue, phiList = le.second_helix(bL, iL, epGuide, epTrue, phiList, hnum, loopFeature, 
                                               xform_True, length_mod=length_mod, distCut=dist_cut)
if verbose:
    print(f'second helix: #{iL.shape[0]}')

if iL.shape[0] == 0:
    if verbose:
        print('fail')
    #return bL, iL, phiList, iL, iL, False

first helix: #3
first loop: #159
second helix: #53


In [48]:
#first next_loop_helix iteration
hnum=1
bL1, iL1, epGuide1, epTrue1, phiList1 = le.next_loop_helix(bL, iL, epGuide, epTrue, phiList, hnum, 
            neighbors=neighbors, phiQueryNum=phiQueryNum, randMult=randMult, distCut=dist_cut,length_mod=length_mod)


In [49]:
bL1.shape

(1039,)

In [50]:
bL1 = copy.deepcopy(bL)
iL1 = iL1.copy()
epGuide1 = epGuide1.copy()
epTrue1 = epTrue1.copy()
phiList1 = phiList1.copy()

In [36]:
#first next_loop_helix iteration
hnum=2
bL2, iL2, epGuide2, epTrue2, phiList2 = le.next_loop_helix(bL1, iL1, epGuide1, epTrue1, phiList1, hnum, 
            neighbors=neighbors, phiQueryNum=phiQueryNum, randMult=randMult, distCut=dist_cut,length_mod=length_mod)

In [37]:
bL2.shape

(4111,)

In [ ]:
bL, iL, epGuide, epTrue, phiList = next_loop_helix(bL, iL, epGuide, epTrue, phiList, x, 
        neighbors=neighbors, phiQueryNum=phiQueryNum, randMult=randMult, distCut=dist_cut,length_mod=length_mod)

In [ ]:


for x in range(hnum,size-1):
    bL, iL, epGuide, epTrue, phiList = next_loop_helix(bL, iL, epGuide, epTrue, phiList, x, 
            neighbors=neighbors, phiQueryNum=phiQueryNum, randMult=randMult, distCut=dist_cut,length_mod=length_mod)

    if iL.shape[0] == 0:
        if verbose:
            print('fail2')
        return bL, iL, phiList, iL, iL, False
    elif iL.shape[0] > 40:
        indexer = random_reduce(bL,num_to_keep = maxPhi_cut)
        bL = bL[indexer]
        iL = iL[indexer] 
        epGuide = epGuide[indexer]
        epTrue =  epTrue[indexer]
        phiList = phiList[indexer]
        if verbose:
            print(f'{x+2} helix: reduced to #{maxPhi_cut}')
    if verbose:
        print(f'{x+2} helix: #{iL.shape[0]}')

#only return proteins with a unique set of loops queried from different phi bins
if uniquePhi:
    phiList, u_indices = np.unique(phiList,axis=0,return_index=True)
    epGuide = epGuide[u_indices]
    epTrue = epTrue[u_indices]
    iL = iL[u_indices]
    bL = bL[u_indices]